In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import precision_score, accuracy_score, f1_score, mean_squared_error
from tqdm import tqdm
import os
os.chdir("D:\\meridianthe4\\PML\\Cases")

In [16]:
hr = pd.read_csv("HR_comma_sep.csv")
X, y = hr.drop("left", axis=1), hr["left"]
ohe = OneHotEncoder(drop='first', sparse_output=False).set_output(transform="pandas")
col_transformer = ColumnTransformer([("OHE", ohe, make_column_selector(dtype_include=object))], 
                                    remainder='passthrough', 
                                    verbose_feature_names_out=False)
col_transformer = col_transformer.set_output(transform="pandas")
X = col_transformer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25, stratify=y)

In [17]:
dtc1 = DecisionTreeClassifier(random_state=25, max_depth=3)
dtc2 = DecisionTreeClassifier(random_state=25, max_depth=2)
lr = LogisticRegression()
voting = VotingClassifier(estimators=[("TREE1", dtc1), ("TREE2", dtc2), ("LR", lr)])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Voting Classifier Accuracy: {accuracy:.4f}")

Voting Classifier Accuracy: 0.9022


d:\meridianthe4\PML\main\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [29]:
sc = StandardScaler()
knn = KNeighborsClassifier(n_neighbors=5)
svm = SVC(kernel='rbf', probability=True, random_state=25)
dct = DecisionTreeClassifier(random_state=25, max_depth=4)
pipeknn = Pipeline([("scaler", sc), ("knn", knn)])
pipesvm = Pipeline([("scaler", sc), ("svm", svm)])
voting = VotingClassifier(estimators=[("KNN", pipeknn), ("SVM", pipesvm), ("DCT", dct)], voting='soft')
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Voting Classifier with Pipelines Accuracy: {accuracy:.4f}")

Voting Classifier with Pipelines Accuracy: 0.9667


In [37]:
depths = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
neighbors = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]
scores = []
for depth in tqdm(depths):
    for neighbor in neighbors:
        dct = DecisionTreeClassifier(random_state=25, max_depth=depth)
        knn = KNeighborsClassifier(n_neighbors=neighbor)
        lr = LogisticRegression(max_iter=5000)
        pipeknn = Pipeline([("scaler", sc), ("knn", knn)])
        voting = VotingClassifier(estimators=[("KNN", pipeknn), ("DCT", dct), ("LR", lr)], voting='soft')
        voting.fit(X_train, y_train)
        y_pred_proba = voting.predict_proba(X_test)
        score = log_loss(y_test, y_pred_proba)
        scores.append([depth, neighbor, score])
df_scores = pd.DataFrame(scores, columns=["Max_Depth", "N_Neighbors", "score"])
df_scores.sort_values("score")

100%|██████████| 10/10 [00:41<00:00,  4.18s/it]


,Max_Depth,N_Neighbors,score
80,18,1,0.133485
60,14,1,0.134499
70,16,1,0.134585
90,20,1,0.134672
50,12,1,0.135912
...,...,...,...
5,2,11,0.237809
6,2,13,0.240461
7,2,15,0.242562
8,2,17,0.244970


In [36]:
sc = StandardScaler()
knn1 = KNeighborsClassifier(n_neighbors=5)
knn2 = KNeighborsClassifier(n_neighbors=3)
lr = LogisticRegression(max_iter=5000)
dct1 = DecisionTreeClassifier(random_state=25, max_depth=4)
dct2 = DecisionTreeClassifier(random_state=25, max_depth=7)
pipeknn1 = Pipeline([("scaler", sc), ("knn", knn1)])
pipeknn2 = Pipeline([("scaler", sc), ("knn", knn2)])
voting = VotingClassifier(estimators=[("KNN1", pipeknn1), ("KNN2", pipeknn2), ("LR", lr), ("DCT1", dct1), ("DCT2", dct2)], voting='soft')
voting.fit(X_train, y_train)
y_pred_proba = voting.predict_proba(X_test)
logLoss = log_loss(y_test, y_pred_proba)
print(f"Voting Classifier with Multiple KNN and DCT Log Loss: {logLoss:.4f}")

Voting Classifier with Multiple KNN and DCT Log Loss: 0.1251
